# Example of using C++ code in Jupyter with the cpt package

It is also possible to use our C++ code from python and Jupyter. This involves using the [SWIG](http://www.swig.org) package. You can download it [here](http://www.swig.org/survey.html) and then install following instructions [here](http://www.swig.org/Doc3.0/Preface.html#Preface_installation). If you are successful, you should be able to open a new terminal and type ```which swig``` to obtain the path of swig. 

The idea is then to use SWIG to automatically generate python-readable code from our "cpt" C++/C libraries. There is a lot to learn in this regard, so we will try first with a simple example that illustrates some concepts we will need, such as using STL libraries and C++11 compilation. 



## Step 1 : Simple example compilation in Terminal

You should be able to see these two simple files: 

#### example.hpp

```cpp
#ifndef EXAMPLE_HPP
#define EXAMPLE_HPP

#include <vector>

// Declare your function normally
int sum_int(std::vector<int> const & vec);

#endif // EXAMPLE_HPP
```

#### example.cpp
```cpp
#include "example.hpp"

// Define your function as you need.
int sum_int(std::vector<int> const & vec)
{
  int sum = 0;
  for ( auto i : vec ) {
    sum += i;
  }
  return sum;
}
```

Notice that there are several advanced features here, including a range-based for loop and the use of ```std::vector```. This simply sums the elements in the loop and returns it. 


### Compile them yourself to get an object file:  

```
g++ --std=c++11 -c example.cpp
```


You should see "example.o" in your directory. 




## Step 2: Creation of SWIG interface file in Terminal 

The magic of SWIG is to create wrapper C++ functions that use the "cython" interface. You will see an "interface" file for SWIG: 

#### example.i
```swig
%module example
/* First: Include your own code.*/
%{
#define SWIG_FILE_WITH_INIT
#include "example.hpp"
%}
/* Next: declare the functions you want to use.*/
int sum_int(std::vector<int> const & vec);

/* Finally: include any other libraries and types you need. 
Yes, you need one type per class, so that means one type per
template instance.*/


%include "std_vector.i"

namespace std {
   %template(vector_int) vector<int>;
};
```


This tells SWIG to import the function ```sum_int``` from the file ```example.hpp```. It also tells SWIG how to use the ```std::vector``` inside of python. 

**IMPORTANT NOTE:** The name you give to the ```std::vector<int>``` is ```vector_int```. This is an alias for the C++ class ```std::vector<int>```, and you are picking the name yourself. The reason this must be done instead of using a python ```list``` object is because python does not have a "native" notion of ```std::vector<int>```. It can convert python lists to C++ arrays, but not to ```std::vector```. To make it more transparent, you could copy the elements into a python list, but that is expensive computationally. You could also convert the ```std::vector``` to a C++ array inside your C++ code, in which case you need not use this. However, for performant usage of ```std::vector``` in python with SWIG, stick to using the implementation from ```std_vector.i```. 


To create the SWIG-generated C++ file, you also need to tell SWIG what to do with the ```setup.py``` file here:

#### setup.py
```python
#!/usr/bin/env python

"""
setup.py file for SWIG example
"""

from distutils.core import setup, Extension


example_module = Extension('_example',
                           sources=['example_wrap.cxx', 'example.cpp'],
                           extra_compile_args=["-std=c++11"],
                           )

setup (name = 'example',
       version = '0.1',
       author      = "SWIG Docs",
       description = """Simple swig example from docs""",
       ext_modules = [example_module],
       py_modules = ["example"],
       )
```


This tells SWIG to generate a file called ```example_wrap.cc``` using source from ```example.cpp``` with C++11. 

### To run this, you do:

```
swig -c++ -python example.i 
```


This will create two files, ```example_wrap.cxx``` and ```example.py```. They are not intended to be human-readable, so we don't need to look into them. 

### Next, we execute 

```
python setup.py build_ext --inplace
```

This will compile our file ```example.cpp``` along with ```example_wrap.cxx``` using the rules we set (for instance, using the C++11 compiler). 

We are now free to use this in python!

 


## Step 3: Make profit: 

First we import the path correctly (we started in our PHY410/jupyter path in this notebook), and check that the path was right.

In [3]:
import sys
import os

sys.path.append( os.path.abspath("swig_example") )

print sys.path




['', '/Library/Python/2.7/site-packages', '/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python27.zip', '/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7', '/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/plat-darwin', '/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/plat-mac', '/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/plat-mac/lib-scriptpackages', '/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/lib-tk', '/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/lib-old', '/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/lib-dynload', '/Users/rappoccio/Library/Python/2.7/lib/python/site-packages', '/Library/Python/2.7/site-packages', '/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python', '/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/PyObjC', '/Library/Python/2.7/si

Next we import our actual function from "example.py"

In [6]:
from example import *


And finally, we can use our code: 

In [7]:
a = vector_int([1,2,3])
x = sum_int(a)
print x


6


### Success!